In [12]:
import pandas as pd
import numpy as np
import os

In [13]:
# metric in long-term-forecasting
# mae, mse, rmse, mape, mspe

In [14]:
pred_lens = [96, 192, 336, 720]
path = '/data/coding/chaochuan/TSGym/resultsGym'
result_dict = {}
for pred_len in pred_lens:
    model_list = os.listdir(path)
    model_list = [_ for _ in model_list if f'pl{pred_len}' in _]
    result_dict[pred_len] = {}
    for model in model_list:
        try:
            result = np.load(os.path.join(path, model, 'metrics.npy'), allow_pickle=True)
            result_dict[pred_len][model] = result[1]
        except:
            pass
            continue

In [15]:
df = pd.DataFrame.from_dict(result_dict[96], orient='index')
df.columns = ['mse']
# df.sort_values(by='mse').to_csv('gym_96.csv', index=True)
df.sort_values(by='mse').head(20)

,mse
long_term_forecast_ECL_96_96_TSGym_RevIN_True_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.176801
long_term_forecast_ECL_96_96_TSGym_RevIN_False_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.177675
long_term_forecast_ECL_96_96_TSGym_RevIN_False_series-patching_Transformer_auto-correlation_True_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.178795
long_term_forecast_ECL_96_96_TSGym_RevIN_False_series-patching_Transformer_self-attention_True_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.179232
long_term_forecast_ECL_96_96_TSGym_RevIN_False_series-patching_Transformer_sparse-attention_True_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.181501
long_term_forecast_ECL_96_96_TSGym_Stat_False_series-patching_Transformer_self-attention_True_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.185608
long_term_forecast_ECL_96_96_TSGym_Stat_False_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.185951
long_term_forecast_ECL_96_96_TSGym_None_False_series-patching_Transformer_self-attention_True_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.186132
long_term_forecast_ECL_96_96_TSGym_None_False_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.186228
long_term_forecast_ECL_96_96_TSGym_RevIN_True_series-patching_Transformer_self-attention_True_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.186368


In [16]:
df = pd.DataFrame.from_dict(result_dict[192], orient='index')
df.columns = ['mse']
df.sort_values(by='mse').head(20)
# df.sort_values(by='mse').to_csv('gym_192.csv', index=True)

,mse
long_term_forecast_ECL_96_192_TSGym_RevIN_False_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl192_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.186802
long_term_forecast_ECL_96_192_TSGym_RevIN_True_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl192_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.187046
long_term_forecast_ECL_96_192_TSGym_RevIN_False_series-patching_Transformer_self-attention_True_custom_ftM_sl96_ll48_pl192_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.187564
long_term_forecast_ECL_96_192_TSGym_RevIN_False_series-patching_Transformer_auto-correlation_True_custom_ftM_sl96_ll48_pl192_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.188862
long_term_forecast_ECL_96_192_TSGym_RevIN_False_series-patching_Transformer_sparse-attention_True_custom_ftM_sl96_ll48_pl192_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.189689
long_term_forecast_ECL_96_192_TSGym_Stat_False_series-patching_Transformer_self-attention_True_custom_ftM_sl96_ll48_pl192_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.190790
long_term_forecast_ECL_96_192_TSGym_None_True_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl192_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.191062
long_term_forecast_ECL_96_192_TSGym_None_False_series-patching_Transformer_self-attention_True_custom_ftM_sl96_ll48_pl192_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.191197
long_term_forecast_ECL_96_192_TSGym_None_False_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl192_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.191283
long_term_forecast_ECL_96_192_TSGym_Stat_False_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl192_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.191710


In [17]:
df = pd.DataFrame.from_dict(result_dict[336], orient='index')
df.columns = ['mse']
df.sort_values(by='mse').head(20)
# df.sort_values(by='mse').to_csv('gym_336.csv', index=True)

,mse
long_term_forecast_ECL_96_336_TSGym_RevIN_False_series-patching_Transformer_self-attention_True_custom_ftM_sl96_ll48_pl336_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.203725
long_term_forecast_ECL_96_336_TSGym_RevIN_False_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl336_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.204156
long_term_forecast_ECL_96_336_TSGym_RevIN_True_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl336_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.204853
long_term_forecast_ECL_96_336_TSGym_RevIN_False_series-patching_Transformer_sparse-attention_True_custom_ftM_sl96_ll48_pl336_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.205823
long_term_forecast_ECL_96_336_TSGym_None_False_series-patching_Transformer_sparse-attention_True_custom_ftM_sl96_ll48_pl336_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.206239
long_term_forecast_ECL_96_336_TSGym_None_True_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl336_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.206388
long_term_forecast_ECL_96_336_TSGym_None_False_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl336_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.206505
long_term_forecast_ECL_96_336_TSGym_RevIN_False_series-patching_Transformer_auto-correlation_True_custom_ftM_sl96_ll48_pl336_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.206900
long_term_forecast_ECL_96_336_TSGym_None_False_series-patching_Transformer_self-attention_True_custom_ftM_sl96_ll48_pl336_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.207091
long_term_forecast_ECL_96_336_TSGym_Stat_False_series-patching_Transformer_self-attention_True_custom_ftM_sl96_ll48_pl336_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.207102


In [18]:
df = pd.DataFrame.from_dict(result_dict[720], orient='index')
df.columns = ['mse']
df.sort_values(by='mse').head(20)
# df.sort_values(by='mse').to_csv('gym_720.csv', index=True)

,mse
long_term_forecast_ECL_96_720_TSGym_None_False_series-patching_Transformer_self-attention_True_custom_ftM_sl96_ll48_pl720_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.240546
long_term_forecast_ECL_96_720_TSGym_None_True_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl720_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.241914
long_term_forecast_ECL_96_720_TSGym_None_False_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl720_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.241936
long_term_forecast_ECL_96_720_TSGym_None_False_series-patching_Transformer_auto-correlation_True_custom_ftM_sl96_ll48_pl720_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.244755
long_term_forecast_ECL_96_720_TSGym_None_True_series-patching_Transformer_sparse-attention_True_custom_ftM_sl96_ll48_pl720_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.244812
long_term_forecast_ECL_96_720_TSGym_RevIN_False_series-patching_Transformer_self-attention_True_custom_ftM_sl96_ll48_pl720_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.246096
long_term_forecast_ECL_96_720_TSGym_None_False_series-patching_Transformer_frequency-enhanced-attention_True_custom_ftM_sl96_ll48_pl720_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.246174
long_term_forecast_ECL_96_720_TSGym_None_False_series-patching_Transformer_sparse-attention_True_custom_ftM_sl96_ll48_pl720_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.246675
long_term_forecast_ECL_96_720_TSGym_RevIN_True_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl720_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.246863
long_term_forecast_ECL_96_720_TSGym_RevIN_False_series-patching_GRU_null_True_custom_ftM_sl96_ll48_pl720_dm512_nh8_el2_dl1_df2048_expand2_dc4_fc3_ebtimeF_dtTrue_Exp_0,0.247005
